# V2 Post-Mortem Diagnostic Experiments
Scientific decomposition: diagnose before fixing. Cheapest experiments first.

In [ ]:
import sys, os, logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, os.path.abspath(".."))

from src.integration import diagnostic_experiments as dexp
from src.integration.notebook_helpers import assemble_full_config, replay_oos_simulation
from src.integration.pipeline_state import load_run_data
from src.config import PipelineConfig

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
logger = logging.getLogger(__name__)
%matplotlib inline

In [ ]:
# === EDIT THESE PATHS ===
RUN_DIR = "results/diagnostic_runs/YYYY-MM-DD_HHMMSS"  # V1 or latest checkpoint
# RUN_DIR_V2 = "results/diagnostic_runs/2026-02-23_233353"  # V2 for comparison

# Phase 0 reverted config (clean baseline)
config = assemble_full_config(
    seed=42,
    n_stocks=1000,  # Default, not 4500
    K=50,           # Default, not 75
    max_epochs=150,
    # Phase 0 reverts:
    lambda_co_max=0.1,
    momentum_enabled=False,
    momentum_weight=0.0,
    phi=5.0,
    sca_tol=1e-5,
)
print(f"Config: N={config.data.n_stocks}, K={config.vae.K}")

In [ ]:
# Load checkpoint data
exp_data = dexp.load_experiment_data(RUN_DIR)
B_A = exp_data["B_A"]
stock_ids = exp_data["stock_ids"]
print(f"Loaded: B_A {B_A.shape}, {len(stock_ids)} stocks, AU={exp_data.get('AU', B_A.shape[1])}")

# Load returns and trailing_vol (same as dashboard.ipynb cells 1-8)
# NOTE: User must provide their own data loading here
# returns = ...  # DataFrame (dates x permnos)
# trailing_vol = ...  # DataFrame (dates x permnos)
# stock_data = ...  # Raw stock data
raise NotImplementedError("Load returns, trailing_vol, stock_data here (same as dashboard.ipynb)")

In [ ]:
# Train/OOS split
all_dates = returns.index
holdout_fraction = 0.2
split_idx = int(len(all_dates) * (1 - holdout_fraction))
train_start = str(all_dates[0].date())
train_end = str(all_dates[split_idx - 1].date())
oos_start = str(all_dates[split_idx].date())
oos_end = str(all_dates[-1].date())
returns_oos = returns.loc[oos_start:oos_end]
print(f"Train: [{train_start}, {train_end}] ({split_idx} days)")
print(f"OOS:   [{oos_start}, {oos_end}] ({len(returns_oos)} days)")

## Tier 0: Component Substitution (~5 min from checkpoint)
Replace ONE component with a "known-good" version to identify the bottleneck.

In [ ]:
tier0_results = dexp.run_all_tier0(
    B_A=B_A, returns=returns, trailing_vol=trailing_vol,
    stock_ids=stock_ids, config=config,
    train_start=train_start, train_end=train_end,
    returns_oos=returns_oos,
)
display(tier0_results.round(4))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart: Sharpe by experiment
ax = axes[0]
valid = tier0_results.dropna(subset=["sharpe"])
colors = ["green" if s > 0 else "red" for s in valid["sharpe"]]
ax.barh(valid["experiment"], valid["sharpe"], color=colors, alpha=0.7)
ax.axvline(0, color="black", linewidth=0.5)
ax.set_xlabel("Sharpe Ratio (OOS)")
ax.set_title("Tier 0: Component Substitution")

# Decision annotations
best_exp = valid.loc[valid["sharpe"].idxmax()]
ax.annotate(f"Best: {best_exp['experiment']}", xy=(best_exp["sharpe"], best_exp["experiment"]),
            fontsize=8, fontweight="bold")

# n_active by experiment
ax = axes[1]
if "n_active" in valid.columns:
    ax.barh(valid["experiment"], valid["n_active"], color="steelblue", alpha=0.7)
    ax.set_xlabel("Number of Active Positions")
    ax.set_title("Portfolio Cardinality")

plt.tight_layout()
plt.show()

In [ ]:
baseline = tier0_results[tier0_results["experiment"] == "T0.0_baseline"]
pca_oracle = tier0_results[tier0_results["experiment"] == "T0.1_pca_oracle"]

baseline_sharpe = float(baseline["sharpe"].iloc[0]) if len(baseline) > 0 else np.nan
pca_sharpe = float(pca_oracle["sharpe"].iloc[0]) if len(pca_oracle) > 0 else np.nan

print("=" * 60)
print("TIER 0 DECISION GATE")
print("=" * 60)
if not np.isnan(pca_sharpe) and pca_sharpe > baseline_sharpe + 0.1:
    print(f"  PCA oracle Sharpe ({pca_sharpe:.3f}) >> baseline ({baseline_sharpe:.3f})")
    print("  => FACTOR MODEL IS THE BOTTLENECK (Category B/E)")
    print("  => Proceed to Tier 1 to confirm, then Tier 3 for retraining")
elif not np.isnan(pca_sharpe) and pca_sharpe < -0.5:
    print(f"  PCA oracle Sharpe ({pca_sharpe:.3f}) still negative")
    print("  => PIPELINE IS ALSO BROKEN (Categories C/D)")
    print("  => Investigate risk model and solver")
else:
    print(f"  Baseline: {baseline_sharpe:.3f}, PCA oracle: {pca_sharpe:.3f}")
    print("  => Mixed signal -- proceed to Tier 1 and 2")

## Tier 1: Factor Quality Profiling (~15 min)
Is the VAE B_A better than random? Compute metrics independent of risk model.

In [ ]:
tier1_results = dexp.run_all_tier1(
    B_A=B_A, returns=returns.loc[train_start:train_end],
    stock_ids=stock_ids, n_random_trials=100, seed=42,
)
print(f"CS R2:              {tier1_results['cs_r2']:.4f} ({tier1_results['cs_r2']*100:.2f}%)")
print(f"Random baseline R2: {tier1_results['random_baseline_r2']:.4f} ({tier1_results['random_baseline_r2']*100:.2f}%)")
print(f"Effective rank:     {tier1_results['effective_rank']:.1f}")
print(f"Condition number:   {tier1_results['condition_number']:.1f}")
print(f"Top-1 eigenvalue:   {tier1_results['top_1_eigenvalue_pct']*100:.1f}%")
print(f"Factor autocorr:    {tier1_results['factor_autocorr_mean']:.3f}")
print(f"\nDECISION: {tier1_results['decision']}")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# CS R2 distribution across dates
ax = axes[0, 0]
r2_by_date = tier1_results["cs_r2_by_date"]
ax.hist(r2_by_date, bins=50, alpha=0.7, label=f"VAE (mean={tier1_results['cs_r2']:.3f})")
ax.axvline(tier1_results["random_baseline_r2"], color="red", linestyle="--",
           label=f"Random baseline ({tier1_results['random_baseline_r2']:.3f})")
ax.set_xlabel("Cross-Sectional R2")
ax.set_ylabel("Count")
ax.set_title("CS R2 Distribution Across Dates")
ax.legend()

# Singular value spectrum
ax = axes[0, 1]
sv = tier1_results["singular_values"]
ax.semilogy(range(1, len(sv)+1), sv, "b.-")
ax.axhline(sv[0] * 0.01, color="red", linestyle="--", alpha=0.5, label="1% of max")
ax.axvline(tier1_results["effective_rank"], color="green", linestyle="--",
           label=f"Effective rank = {tier1_results['effective_rank']:.1f}")
ax.set_xlabel("Component Index")
ax.set_ylabel("Singular Value")
ax.set_title("B_A Singular Value Spectrum")
ax.legend()

# Factor autocorrelation
ax = axes[1, 0]
autocorr = tier1_results["factor_autocorr"]
ax.bar(range(len(autocorr)), autocorr, alpha=0.7)
ax.set_xlabel("Factor Index")
ax.set_ylabel("Autocorrelation (lag-1)")
ax.set_title(f"Factor z_hat Autocorrelation (mean |rho|={tier1_results['factor_autocorr_mean']:.3f})")
ax.axhline(0, color="black", linewidth=0.5)

# Eigenvalue concentration
ax = axes[1, 1]
sv_sq = sv ** 2
sv_cumsum = np.cumsum(sv_sq) / sv_sq.sum()
ax.plot(range(1, len(sv_cumsum)+1), sv_cumsum, "b.-")
ax.axhline(0.5, color="red", linestyle="--", alpha=0.5, label="50%")
ax.axhline(0.9, color="orange", linestyle="--", alpha=0.5, label="90%")
ax.set_xlabel("Number of Components")
ax.set_ylabel("Cumulative Variance Explained")
ax.set_title("Eigenvalue Concentration")
ax.legend()

plt.tight_layout()
plt.show()

## Tier 2: Parameter Sensitivity (~10 min total)
Sweep ONE parameter at a time, all others at Phase 0 values.

In [ ]:
tier2_results = dexp.run_all_tier2(
    B_A=B_A, returns=returns, trailing_vol=trailing_vol,
    stock_ids=stock_ids, config=config,
    train_start=train_start, train_end=train_end,
    returns_oos=returns_oos,
)
for name, df in tier2_results.items():
    print(f"\n--- {name} ---")
    display(df.round(4))

In [ ]:
n_sweeps = len(tier2_results)
fig, axes = plt.subplots(1, n_sweeps, figsize=(5*n_sweeps, 4))
if n_sweeps == 1:
    axes = [axes]

for ax, (name, df) in zip(axes, tier2_results.items()):
    valid = df.dropna(subset=["sharpe"])
    if len(valid) == 0:
        continue
    x_labels = [str(v) for v in valid["param_value"]]
    ax.bar(x_labels, valid["sharpe"], alpha=0.7)
    ax.set_xlabel(name)
    ax.set_ylabel("Sharpe")
    ax.set_title(f"Sweep: {name}")
    ax.axhline(0, color="black", linewidth=0.5)
    # Mark best
    best_idx = valid["sharpe"].idxmax()
    ax.bar(x_labels[list(valid.index).index(best_idx)],
           valid.loc[best_idx, "sharpe"], color="green", alpha=0.9)

plt.tight_layout()
plt.show()

In [ ]:
print("=" * 60)
print("TIER 2: BEST DOWNSTREAM CONFIG")
print("=" * 60)
for name, df in tier2_results.items():
    valid = df.dropna(subset=["sharpe"])
    if len(valid) == 0:
        print(f"  {name}: ALL FAILED")
        continue
    best = valid.loc[valid["sharpe"].idxmax()]
    print(f"  {name}: best value = {best['param_value']}, Sharpe = {best['sharpe']:.3f}")

## Tier 3: Training Experiments (3-15h on Colab)
Based on Tier 0-2 results, design targeted retraining experiments.
Each tests ONE hypothesis. Run on Colab with GPU.

In [ ]:
# T3.1: N=1000, K=50 (addresses biggest blind spot)
# Apply best Tier 2 params from above
config_t31 = assemble_full_config(
    seed=42,
    n_stocks=1000,
    K=50,
    max_epochs=150,
    # Phase 0 reverts:
    lambda_co_max=0.1,
    momentum_enabled=False,
    momentum_weight=0.0,
    phi=5.0,
    sca_tol=1e-5,
    # Apply Tier 2 best params here:
    # sigma_z_shrinkage=...,
    # disable_vt=...,
)
print(f"T3.1 Config: N={config_t31.data.n_stocks}, K={config_t31.vae.K}, "
      f"epochs={config_t31.training.max_epochs}")
print("Run this config in dashboard.ipynb on Colab (expected ~2-3h)")

In [ ]:
# T3.2: N=4500, K=50 (isolates K effect)
config_t32 = assemble_full_config(seed=42, n_stocks=4500, K=50, max_epochs=150,
    lambda_co_max=0.1, momentum_enabled=False, phi=5.0, sca_tol=1e-5)

# T3.3: N=1000, K=75 (isolates N effect)
config_t33 = assemble_full_config(seed=42, n_stocks=1000, K=75, max_epochs=150,
    lambda_co_max=0.1, momentum_enabled=False, phi=5.0, sca_tol=1e-5)

print("Factorial design:")
print("         K=50      K=75")
print(f"N=1000   T3.1      T3.3")
print(f"N=4500   T3.2      (V1=existing)")
print("\nOnly run T3.2/T3.3 if T3.1 results are ambiguous")

In [ ]:
# After running T3.1 (and optionally T3.2, T3.3) on Colab:
# T3_1_RUN_DIR = "results/diagnostic_runs/YYYY-MM-DD_HHMMSS"
# t3_1_data = dexp.load_experiment_data(T3_1_RUN_DIR)
# t3_1_tier1 = dexp.run_all_tier1(t3_1_data["B_A"], returns, t3_1_data["stock_ids"])
# print(f"T3.1 CS R2: {t3_1_tier1['cs_r2']*100:.2f}%, Decision: {t3_1_tier1['decision']}")
print("TODO: Fill in after running T3 experiments on Colab")

## Decision & Summary

In [ ]:
# Combine all results into one table
all_experiments = tier0_results.copy()
# Add tier2 best configs if available
for name, df in tier2_results.items():
    valid = df.dropna(subset=["sharpe"])
    if len(valid) > 0:
        best = valid.loc[valid["sharpe"].idxmax()].to_dict()
        best["experiment"] = f"T2_best_{name}"
        all_experiments = pd.concat([all_experiments, pd.DataFrame([best])], ignore_index=True)

display(all_experiments.sort_values("sharpe", ascending=False).round(4))

In [ ]:
# Diagnostic scorecard
print("=" * 70)
print("DIAGNOSTIC SCORECARD")
print("=" * 70)
print(f"{'Category':<20} {'Metric':<30} {'Value':<15} {'Status':<10}")
print("-" * 70)

cs_r2 = tier1_results['cs_r2']
eff_rank = tier1_results['effective_rank']
cond = tier1_results['condition_number']
baseline_row = tier0_results[tier0_results["experiment"] == "T0.0_baseline"].iloc[0]

def status(val, good_lo, good_hi, crit_lo=None, crit_hi=None):
    if good_lo <= val <= good_hi:
        return "OK"
    return "CRITICAL"

print(f"{'Factor Quality':<20} {'CS R2 (%)':<30} {cs_r2*100:<15.2f} {status(cs_r2, 0.15, 1.0):<10}")
print(f"{'':<20} {'Effective rank':<30} {eff_rank:<15.1f} {status(eff_rank, 10, 1000):<10}")
print(f"{'':<20} {'Random baseline R2':<30} {tier1_results['random_baseline_r2']*100:<15.2f} {'OK' if tier1_results['random_baseline_r2'] < cs_r2 else 'CRITICAL':<10}")
print(f"{'Risk Model':<20} {'Condition number':<30} {cond:<15.1f} {status(cond, 0, 100):<10}")
print(f"{'Portfolio':<20} {'Sharpe (OOS)':<30} {baseline_row['sharpe']:<15.3f} {status(baseline_row['sharpe'], 0.3, 10):<10}")
print(f"{'':<20} {'Max drawdown':<30} {baseline_row.get('max_drawdown', 1.0):<15.3f} {status(baseline_row.get('max_drawdown', 1.0), 0, 0.5):<10}")

In [ ]:
import json
from pathlib import Path

output_dir = Path("results/experiments")
output_dir.mkdir(parents=True, exist_ok=True)

# Save tier 0
tier0_results.to_csv(output_dir / "tier0_results.csv", index=False)

# Save tier 1
tier1_export = {k: v for k, v in tier1_results.items()
                if not isinstance(v, (np.ndarray, list)) or (isinstance(v, list) and len(v) < 100)}
with open(output_dir / "tier1_results.json", "w") as f:
    json.dump(tier1_export, f, indent=2, default=str)

# Save tier 2
for name, df in tier2_results.items():
    df.to_csv(output_dir / f"tier2_{name}.csv", index=False)

print(f"Results exported to {output_dir}/")